In [26]:
using BenchmarkTools
using Yao, FLOYao
using LinearAlgebra

N = 30

nq = N
dim = 2 * nq
layers = 2
g = chain(nq)
for _ in 1:layers
    for i in 1:nq-1
        push!(g, rot(kron(nq, i => X, i+1 => X), 0.))
    end
    for i in 1:nq-1
        push!(g, rot(kron(nq, i => X, i+1 => Y), 0.))
    end
    for i in 1:nq
        push!(g, put(nq, i => Rz(0.)))
    end
end

⊗ = kron

function covariance_matrix(reg::MajoranaReg)
    nq = nqubits(reg)
    G = I(nq) ⊗ [0 1; -1 0]
    return reg.state * G * reg.state'
end

function helper!(b, temp_m, probabilities) #Evolves all matrices and probabilities and gradients by nq steps, in-place and optimally
    nq = N
    dim = 2*nq
    for i in 2:nq
        i1 = 2*(i-1)
        ni = b[i-1]
        cur_prob = probabilities[i-1]
        cur_prefactor = (-1)^ni / (2*cur_prob)
        for p in 2*(i-1)+1:dim
            for q in p+1:dim
                temp_m[p,q] -= cur_prefactor * (temp_m[i1-1,p] * temp_m[i1,q])
                temp_m[p,q] += cur_prefactor * (temp_m[i1-1,q] * temp_m[i1,p])
            end
        end
        ni = b[i]
        probabilities[i] = (1+(-1)^ni * temp_m[i1+1, i1+2]) / 2
    end
end

function update_opt!(reg::MajoranaReg, theta, b, temp_m, probabilities) 
    nq = nqubits(reg)
    nparams = nparameters(g)
    dim = 2*nq
    t_tot = 0
    for i in 1:2
        if i > 1
            t = time()
            helper!(b, temp_m, probabilities)
            diff = (time() - t) * 10^6
            println("iteration $i : $diff")
        else
            i1 = 0
            dispatch!(g, theta)
            z = reg |> g
            temp_m = covariance_matrix(z)
            ni = b[i]
            probabilities[i] = (1+(-1)^ni * temp_m[i1+1, i1+2]) / 2
        end
    end
end

#Setting g to have random parameters, then doing a measurement. The result is stored in 'bitstr'
p = rand(nparameters(g)).*2π
dispatch!(g, p)
println("number of parameters: ", nparameters(g))
reg = FLOYao.zero_state(nq) |> g
results = measure(reg, nshots = 1)
bitstr = results[1]
println("measured outcome: $bitstr")
println("probability of measuring the above outcome: ", FLOYao.bitstring_probability(reg, bitstr))

#Initializing temporary matrix and vector, used in the algorithm
temp_m = Matrix{Float64}(undef, dim, dim)
probabilities = Vector{Float64}(undef, nq)

println("the time (μs) taken for 'iteration 2' represents the time needed to compute p_θ(x_i|x_1,...x_{i-1}) and ∇_θ(p_θ(x_i|x_1,...x_{i-1})) for i=2,...,nq")
update_opt!(FLOYao.zero_state(N), p, bitstr, temp_m, probabilities)

number of parameters: 236
measured outcome: 1100101001000000100011110111010011011101 ₍₂₎
probability of measuring the above outcome: 1.596690509997557337980125680966852750169344964609780273425357559164395674523522e-10
the time (μs) taken for 'iteration 2' represents the time needed to compute p_θ(x_i|x_1,...x_{i-1}) and ∇_θ(p_θ(x_i|x_1,...x_{i-1})) for i=2,...,nq
iteration 2 : 93188.04740905762


In [27]:
temp_m = Matrix{Float64}(undef, dim, dim)
probabilities = Vector{Float64}(undef, nq)
update_opt!(FLOYao.zero_state(N), p, bitstr, temp_m, probabilities)

iteration 2 : 76686.85913085938


In [28]:
temp_m = Matrix{Float64}(undef, dim, dim)
probabilities = Vector{Float64}(undef, nq)
update_opt!(FLOYao.zero_state(N), p, bitstr, temp_m, probabilities)

iteration 2 : 67372.79891967773


In [29]:
temp_m = Matrix{Float64}(undef, dim, dim)
probabilities = Vector{Float64}(undef, nq)
update_opt!(FLOYao.zero_state(N), p, bitstr, temp_m, probabilities)
println("Running update_opt! repeatedly seems to show that the first time is very slow but subsequent times are much faster.")

iteration 2 : 70769.07157897949
Running update_opt! repeatedly seems to show that the first time is very slow but subsequent times are much faster.


In [30]:
temp_m = Matrix{Float64}(undef, dim, dim)
prob = Vector{Float64}(undef, N) 
prob[1] = rand() #Randomly assign a number to p(x_1). A vector with nothing except a float in the first entry is what gets passed into the helper! function, in the update_opt! function.
@benchmark helper!(bitstr, temp_m, prob)

BenchmarkTools.Trial: 125 samples with 1 evaluation.
 Range (min … max):  37.087 ms … 57.663 ms  ┊ GC (min … max): 0.00% … 6.44%
 Time  (median):     39.872 ms              ┊ GC (median):    5.58%
 Time  (mean ± σ):   40.293 ms ±  2.952 ms  ┊ GC (mean ± σ):  3.16% ± 3.06%

   █▃ ▁   ▃ ▄   ▄  ▂                                           
  ▆██▇██▆██████▁█▇██▄▃▇▆▄▃▃▁▁▁▃▁▄▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▃▃▁▁▁▁▁▁▁▁▁▁▃ ▃
  37.1 ms         Histogram: frequency by time        52.3 ms <

 Memory estimate: 22.28 MiB, allocs estimate: 689897.